In [37]:
import numpy as np
import sys
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, Input
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import load_model
import tensorflow.keras.losses
from keras.utils import to_categorical

from constants import * 
from heuristic import *
from io_help import *
from solver import *

def one_hot_encode(board):
    """ 
    This function one hot encodes the board into a length 256 array.
    The one hot encoding gives the location of each number in the board.
    For example, the first 16 of the 256 numbers will indicate where on
    the board the 1 tile is. 
    """

    flat = (board.reshape(SIZE ** 2)).tolist()

    X = []
    for i in np.arange(1,17): 
        encoding = np.zeros(SIZE ** 2)
        encoding[flat.index(i)] = 1

        X.append(encoding)  

    X = (np.asarray(X).reshape(SIZE ** 4))

    # Potentially append Manhattan distance. 
    # np.append(X, manhattan(board))

    return X

def load_data(file_name):
    """
    This function reads in training data from a file and returns 
    the one-hot encoded data X and their labels Y as a tuple. 
    """
    file = open(file_name, "r")

    X = []
    Y = []

    for string in file: 
        (board, classification) = string_to_board_and_dist(string) 

        X.append(one_hot_encode(board))
        Y.append(classification)

    file.close()

    return(np.asarray(X),np.asarray(Y))

def evaluate(X,Y):
    """
    This function reads in training data from a file and 
    trains and evaluates NN model using kfold validation. 
    """
    #(X,Y) = load_data(file_name)

    #Y = to_categorical(Y)

    # Implement K-fold cross validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=2020)

    for train, test in kfold.split(X, Y):
        # Build Model
        model = Sequential()

        # Input Layer
        i = Input(shape = (256,))
        x_1 = Dense(256, activation='relu')(i)
        #x_2 = Dropout(0.1)(x_1)
        x_3 = Dense(128, activation='relu')(x_1)
        #x_4 = Dropout(0.1)(x_3)
        o = Dense(5, activation='softmax')(x_3)
        model = Model(i,o)

        # Define the optimizer and loss function
        model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])

        # Train 
        model.fit(X, Y, epochs=15, verbose=1)

        # Evaluate
        score = model.evaluate(X[test], Y[test], verbose=0)
        print(score)

    return model

def train(X,Y):
    """
    This function reads in training data from a file and returns a 
    trained NN model. 
    """
    #(X,Y) = load_data(file_name)

    #Y = to_categorical(Y)

    # Build Model
    model = Sequential()

    # Input Layer
    i = Input(shape = (256,))
    x_1 = Dense(256, activation='relu')(i)
    #x_2 = Dropout(0.1)(x_1)
    x_3 = Dense(128, activation='relu')(x_1)
    #x_4 = Dropout(0.1)(x_3)
    o = Dense(5, activation='softmax')(x_3)
    model = Model(i,o)

    # Define the optimizer and loss function
    model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])

    # Train 
    model.fit(X, Y, epochs=50, verbose=1)

    return model

In [35]:
(X,Y) = load_data("final_portfolio_data2.txt")

In [ ]:
model = train(X,Y)

board = gen_board()
print(board)
print(model.predict(one_hot_encode(board).reshape(1,256)))

Train on 395715 samples
Epoch 1/75
395715/395715 [==============================] - 49s 124us/sample - loss: 0.6804 - accuracy: 0.7105
Epoch 2/75
395715/395715 [==============================] - 39s 98us/sample - loss: 0.6258 - accuracy: 0.7295 - loss:
Epoch 3/75
395715/395715 [==============================] - 31s 77us/sample - loss: 0.5984 - accuracy: 0.7400
Epoch 4/75
395715/395715 [==============================] - 32s 81us/sample - loss: 0.5778 - accuracy: 0.7492
Epoch 5/75
395715/395715 [==============================] - 30s 75us/sample - loss: 0.5589 - accuracy: 0.7574
Epoch 6/75
395715/395715 [==============================] - 29s 73us/sample - loss: 0.5435 - accuracy: 0.7652
Epoch 7/75
395715/395715 [==============================] - 31s 77us/sample - loss: 0.5295 - accuracy: 0.7717
Epoch 8/75
395715/395715 [==============================] - 31s 78us/sample - loss: 0.5167 - accuracy: 0.7770
Epoch 9/75
395715/395715 [==============================] - 29s 73us/sample - loss: 0.5

In [4]:
model.save("./Algorithm_Models/algorithm_model_75_new_complex.h5")

In [3]:
model2 = load_model("./Algorithm_Models/algorithm_model_75_new_complex.h5")
model2.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256)]             0         
_________________________________________________________________
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1285      
Total params: 67,077
Trainable params: 67,077
Non-trainable params: 0
_________________________________________________________________


In [31]:
board = gen_board()
print(board)
print(np.asarray(model2.predict(one_hot_encode(board).reshape(1,256)).flatten()))
print(np.asarray(model2.predict(one_hot_encode(board).reshape(1,256)).flatten()).argmax())

[[16  1  2 12]
 [ 5 10  6  4]
 [ 9  8  3  7]
 [13 14 11 15]]
[0.26458484 0.64832425 0.05417242 0.03118835 0.00173012]
1
